# Sprint 2: Unsupervised learning

Tijdens deze tweede sprint wordt gefocust op het clusteren van de data. Aan de hand van de verschillende klachtbeschrijvingen trachten we de verschillende categorieën voor de producten en issues te voorspellen zonder de categorieën opvoorhand te specifiëren (unsupervised learning). Op die manier trachten we de klachten op een betere manier te groeperen in vergelijking met de reeds gedefinieërde categorieën in de dataset.

## Exploratory Data Analysis

De data exploratie wort hier beperkt tot enkel het inlezen van de vereiste data. Dit werd reeds tijdens sprint 1 uitvoerig behandeld.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import re, string

rows_to_read = None # Gewenst aantal rijen inlezen (None = allemaal)

complaints = pd.read_csv('../data/complaints.csv', delimiter=',', nrows=rows_to_read)

rows, cols = complaints.shape